In [123]:
import pandas as pd

# Define the condition for filtering (stations you're interested in)
column_name = 'station_number'  # Replace with your column name
value_conditions = [66062, 66037, 67105]  # List of station numbers to include (Observatory Hill, Sydney Airport, and Richmond)

# Define the columns to read (make sure to include 'station_number' in col_to_include)
col_to_include = ['station_number', 'datetime', 'air_temperature_in_degrees_c', 'wind_speed_in_km_h', 'relative_humidity_in_percentage']

# Specify the chunk size (e.g., 10000 rows per chunk)
chunk_size = 10000

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Iterate over the file in chunks
for chunk in pd.read_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\1. raw\BOM Weather Data\bom_data_nsw55167608.csv', 
                         usecols=col_to_include, chunksize=chunk_size):
    # Filter rows where 'station_number' is in the list of value_conditions
    filtered_chunk = chunk[chunk[column_name].isin(value_conditions)]
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks into a single DataFrame
filtered_df = pd.concat(filtered_chunks)

# Display the first 5 filtered rows
print(filtered_df.head(5))


         station_number             datetime  air_temperature_in_degrees_c  \
1747282           66037  2022-12-01 00:00:00                          18.4   
1747283           66037  2022-12-01 00:30:00                          18.2   
1747284           66037  2022-12-01 01:00:00                          18.3   
1747285           66037  2022-12-01 01:30:00                          18.2   
1747286           66037  2022-12-01 02:00:00                          18.1   

         relative_humidity_in_percentage  wind_speed_in_km_h  
1747282                             63.0                13.0  
1747283                             65.0                13.0  
1747284                             66.0                13.0  
1747285                             68.0                13.0  
1747286                             69.0                13.0  


In [ ]:
filtered_df['datetime'] = pd.to_datetime(filtered_df['datetime'], format='%Y-%m-%d %H:%M:%S')
#filter the datetime to include only from 1 July 2010 00:00:00 am to 30 June 2013 23:30:00, which is ASHD data
start_date = '2010-07-01 00:00:00'
end_date = '2013-06-30 23:30:00'
filtered_df = filtered_df[(filtered_df['datetime'] >= start_date) & (filtered_df['datetime'] <= end_date)]

filtered_df_2 = filtered_df[filtered_df['station_number'] == 66037] #use Sydney Airport station only

# make datetime the index
filtered_df_2.set_index('datetime', inplace=True)
#sort the index
filtered_df_2.sort_index(inplace=True)

# create a column of datetime from 1 July 2010 to 30 June 2013 with 30 minute intervals
expected_date_range = pd.date_range(start=start_date, end=end_date, freq='30min')

# make weather data df with the expected date range
weather_data_df = pd.DataFrame(index=expected_date_range)

# set  index name to 'datetime'
weather_data_df.index.name = 'datetime'

# merge the weather data with the filtered data, keeping both datetime from the weather data and the filtered data
weather_data_df = weather_data_df.merge(filtered_df_2, left_index=True, right_index=True, how='outer')

# fill NaN values the nearest previous value
weather_data_df.fillna(method='ffill', inplace=True)

# include only the row with expected date range
weather_data_df = weather_data_df.loc[expected_date_range]

#delete station_number column
weather_data_df.drop(columns=['station_number'], inplace=True)

#delete rows with duplicate datetime index
weather_data_df = weather_data_df[~weather_data_df.index.duplicated(keep='first')]

weather_data_df.to_csv(r'c:\Users\z5404477\OneDrive - UNSW\04_Workspace\2. WIP\data\2. processed\ahsd_weather_data.csv')

# make datetime column the index on ds1 dataframe
ds1.index = ds1['datetime']

ds1.drop(columns=['datetime'], inplace=True)

#convert the index to datetime
ds1.index = pd.to_datetime(ds1.index, format='%Y-%m-%d %H:%M:%S')

#merge the ds1 dataframe with the weather_data_df dataframe
ds4 = ds1.merge(weather_data_df, left_index=True, right_index=True, how='outer')
ds4.to_csv('../../data/ds4_ashd_with_weather.csv')


In [2]:
import pandas as pd
ds4 = pd.read_csv('../../data/ds4_ashd_with_weather.csv', index_col='datetime', parse_dates=True)

In [3]:
ds4

,netload_kW,air_temperature_in_degrees_c,relative_humidity_in_percentage,wind_speed_in_km_h
datetime,,,,
2010-07-01 00:00:00,215.792,8.4,66.0,13.0
2010-07-01 00:30:00,193.348,7.9,67.0,13.0
2010-07-01 01:00:00,166.112,7.5,68.0,16.6
2010-07-01 01:30:00,153.506,7.4,69.0,18.4
2010-07-01 02:00:00,140.508,7.2,68.0,20.5
...,...,...,...,...
2013-06-30 21:30:00,321.514,12.4,88.0,14.8
2013-06-30 22:00:00,309.346,12.3,85.0,16.6
2013-06-30 22:30:00,290.918,12.7,84.0,11.2
